In [ ]:
!pip install transformers

In [ ]:
import os
import cv2
from tqdm import tqdm
from google.colab.patches import cv2_imshow
import requests
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from transformers import YolosImageProcessor, YolosForObjectDetection
from PIL import Image
import torch
import requests

In [ ]:
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")

config.json:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/26.0M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/275 [00:00<?, ?B/s]

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [ ]:
!rm -rf YOLO_bounding_box_images
!rm -rf YOLO_output

In [ ]:
!mkdir YOLO_bounding_box_images
!mkdir YOLO_output

mkdir: cannot create directory ‘YOLO_bounding_box_images’: File exists


In [ ]:
!unzip accident_frames.zip

Archive:  accident_frames.zip
   creating: accident_frames/
   creating: accident_frames/CLIP_183/
  inflating: __MACOSX/accident_frames/._CLIP_183  
   creating: accident_frames/CLIP_43/
  inflating: __MACOSX/accident_frames/._CLIP_43  
   creating: accident_frames/CLIP_88/
  inflating: __MACOSX/accident_frames/._CLIP_88  
   creating: accident_frames/CLIP_28/
  inflating: __MACOSX/accident_frames/._CLIP_28  
   creating: accident_frames/CLIP_311/
  inflating: __MACOSX/accident_frames/._CLIP_311  
   creating: accident_frames/CLIP_19/
  inflating: __MACOSX/accident_frames/._CLIP_19  
   creating: accident_frames/CLIP_124/
  inflating: __MACOSX/accident_frames/._CLIP_124  
   creating: accident_frames/CLIP_26/
  inflating: __MACOSX/accident_frames/._CLIP_26  
   creating: accident_frames/CLIP_89/
  inflating: __MACOSX/accident_frames/._CLIP_89  
   creating: accident_frames/CLIP_20/
  inflating: __MACOSX/accident_frames/._CLIP_20  
   creating: accident_frames/CLIP_298/
  inflating: __

In [ ]:
directory_path = '/content/accident_frames/'
directory_files = os.listdir(directory_path)

# Use a default font
font = ImageFont.load_default()

for video in directory_files:
  print(video)
  if not video.startswith('.'):
    try:
      os.mkdir("/content/YOLO_bounding_box_images/" + video.split(".")[0])
    except OSError as error:
      print(error)
    list_of_objects = []

    image_files = os.listdir(directory_path + "/" + video + "/")
    for image in image_files:
        image_path = os.path.join(directory_path + "/" + video + "/", image)
        pil_image = Image.open(image_path)
        object_count = {}
        device = "cuda" if torch.cuda.is_available() else "cpu"
        # Make a copy of the image to draw bounding boxes without affecting the original clarity
        image_with_boxes = pil_image.copy()
        inputs = image_processor(images=pil_image, return_tensors="pt")
        outputs = model(**inputs)

        # model predicts bounding boxes and corresponding COCO classes
        logits = outputs.logits
        bboxes = outputs.pred_boxes


        # print results
        target_sizes = torch.tensor([pil_image.size[::-1]])
        results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]
        for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
            box = [round(i, 2) for i in box.tolist()]
            print(
                f"Detected {model.config.id2label[label.item()]} with confidence "
                f"{round(score.item(), 3)} at location {box}"
            )
            predicted_label = model.config.id2label[label.item()]
            if(round(score.item(), 3) > 0.65):
                if predicted_label not in object_count:
                  object_count[predicted_label] = 0
                object_count[predicted_label] = object_count[predicted_label] + 1
            bbox = [int(coord) for coord in box]
            # Add a bounding box to the image
            draw = ImageDraw.Draw(image_with_boxes)
            draw.rectangle(bbox, outline='red', width=2)

            text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
            draw.rectangle(
                (bbox[0], bbox[1] - text_height, bbox[0] + text_width, bbox[1]),
                fill='red'
            )
            draw.text((bbox[0], bbox[1] - text_height), model.config.id2label[label.item()], fill='white', font=font)

            Y=bbox[1]
            X=bbox[0]
            # Crop the image using the bounding box coordinates
            #cropped_image_pil = pil_image.crop((X, Y, bbox[2], bbox[3]))
            #cropped_image_pil.save("/content/images_cropped/" + video.split('.')[0] + "_cropped_image_"+ image.split(".")[0] +".png")
        # Convert the PIL Image back to a NumPy array
        image_with_boxes.save("/content/YOLO_bounding_box_images/" + video.split('.')[0] + "/" + image.split(".")[0] + ".png")
        list_of_objects.append({image.split(".")[0]:object_count})
    print(list_of_objects)
    f = open("/content/YOLO_output/" + video.split(".")[0] + ".txt", "w")
    f.write(str(list_of_objects))
    f.close()

CLIP_127
[Errno 17] File exists: '/content/YOLO_bounding_box_images/CLIP_127'
Detected car with confidence 0.917 at location [526.46, 361.36, 564.88, 384.27]
Detected car with confidence 0.961 at location [630.33, 363.74, 714.65, 402.76]


<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


Detected person with confidence 0.992 at location [911.12, 243.28, 980.87, 381.38]
Detected car with confidence 0.903 at location [494.49, 363.98, 535.81, 385.49]
Detected car with confidence 0.993 at location [589.25, 367.55, 681.29, 406.7]


<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


Detected car with confidence 0.949 at location [529.64, 370.46, 588.78, 400.39]
Detected traffic light with confidence 0.985 at location [379.2, 334.9, 403.57, 362.59]
Detected car with confidence 0.965 at location [537.71, 370.56, 631.43, 405.24]


...

Detected car with confidence 0.97 at location [555.45, 361.68, 625.86, 403.25]
Detected car with confidence 0.928 at location [0.33, 525.49, 1280.33, 713.66]
Detected car with confidence 0.964 at location [648.52, 358.28, 732.13, 401.07]


<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)
<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


Detected car with confidence 0.966 at location [612.9, 363.48, 678.42, 397.55]


<ipython-input-27-e399afaa81b4>:51: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_width, text_height = draw.textsize(model.config.id2label[label.item()], font=font)


[{'frame60': {'car': 4}}, {'frame45': {'car': 2}}, {'frame30': {'car': 3}}, {'frame15': {'car': 1}}]


In [ ]:
!zip -r /content/YOLO_bounding_box_images.zip /content/YOLO_bounding_box_images
!zip -r /content/YOLO_output.zip /content/YOLO_output

  adding: content/YOLO_bounding_box_images/ (stored 0%)
  adding: content/YOLO_bounding_box_images/CLIP_127/ (stored 0%)
  adding: content/YOLO_bounding_box_images/CLIP_127/frame60.png (deflated 0%)
  adding: content/YOLO_bounding_box_images/CLIP_127/frame45.png (deflated 0%)
  adding: content/YOLO_bounding_box_images/CLIP_127/frame15.png (deflated 0%)
  adding: content/YOLO_bounding_box_images/CLIP_127/frame30.png (deflated 0%)
  adding: content/YOLO_bounding_box_images/CLIP_308/ (stored 0%)
  adding: content/YOLO_bounding_box_images/CLIP_308/frame29.png (deflated 1%)
  adding: content/YOLO_bounding_box_images/CLIP_308/frame58.png (deflated 0%)
  adding: content/YOLO_bounding_box_images/CLIP_169/ (stored 0%)
  adding: content/YOLO_bounding_box_images/CLIP_169/frame60.png (deflated 0%)
  adding: content/YOLO_bounding_box_images/CLIP_169/frame45.png (deflated 0%)
  adding: content/YOLO_bounding_box_images/CLIP_169/frame15.png (deflated 0%)
  adding: content/YOLO_bounding_box_images/CLIP